In [1]:
!pip install sentencepiece

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Désactiver les avertissements TensorFlow
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)  # Désactiver les avertissements Transformers

import json
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Vérifier si SentencePiece est installé
try:
    import sentencepiece
    print(f"SentencePiece version: {sentencepiece.__version__}")
except ImportError:
    print("SentencePiece n'est pas installé. Veuillez l'installer avec 'pip install sentencepiece'.")
    exit(1)

# Charger les données clusterisées
def load_clustered_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Charger le modèle et le tokenizer PEGASUS
def load_pegasus_model():
    try:
        # Charger le modèle pré-entraîné PEGASUS et le tokenizer
        model_name = "google/pegasus-xsum"  # Modèle PEGASUS pour le résumé abstrait
        tokenizer = PegasusTokenizer.from_pretrained(model_name)
        model = PegasusForConditionalGeneration.from_pretrained(model_name)
        return tokenizer, model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle PEGASUS : {e}")
        return None, None

# Fonction pour générer un résumé avec PEGASUS
def generate_summary_pegasus(text, tokenizer, model, max_length=130, min_length=30):
    try:
        # Tokeniser le texte d'entrée
        inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)
        
        # Générer le résumé
        summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, num_beams=5, early_stopping=True)
        
        # Décoder le résumé en texte
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        print(f"Erreur lors de la génération du résumé avec PEGASUS : {e}")
        return "Résumé non disponible"

# Chemin vers le fichier clusterisé
clustered_file_path = 'clustered_articles2.json'
clustered_data = load_clustered_data(clustered_file_path)

# Regrouper les articles par cluster
clusters = {}
for article in clustered_data:
    cluster_id = article['cluster']
    if cluster_id not in clusters:
        clusters[cluster_id] = []
    clusters[cluster_id].append(article['content'])

# Charger le modèle et le tokenizer PEGASUS
tokenizer, model = load_pegasus_model()
if tokenizer is None or model is None:
    print("Impossible de charger le modèle PEGASUS. Vérifiez les erreurs ci-dessus.")
    exit(1)

# Générer un résumé pour chaque cluster
cluster_summaries = {}
for cluster_id, contents in clusters.items():
    # Concaténer le contenu de tous les articles du cluster
    combined_content = " ".join(contents)
    
    # Limiter la longueur du texte pour respecter les limites du modèle
    max_input_length = 1024  # Limite de tokens pour PEGASUS
    if len(combined_content) > max_input_length:
        combined_content = combined_content[:max_input_length]
    
    # Générer un résumé avec PEGASUS
    summary = generate_summary_pegasus(combined_content, tokenizer, model, max_length=130, min_length=30)
    cluster_summaries[cluster_id] = summary

# Sauvegarder les résumés dans un fichier JSON
output_file_path = 'cluster_summaries_pegasus.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(cluster_summaries, file, ensure_ascii=False, indent=4)

print(f"Les résumés ont été enregistrés dans '{output_file_path}'.")

SentencePiece version: 0.2.0
Les résumés ont été enregistrés dans 'cluster_summaries_pegasus.json'.
